<a href="https://colab.research.google.com/github/MohamedWael45/RFM/blob/main/RFM_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import os
from sklearn.cluster import KMeans
url="https://raw.githubusercontent.com/MohamedWael45/RFM/main/RFMwithExcel.xlsx"
df=pd.read_excel(url)


In [28]:
df_second_page = pd.read_excel(url, sheet_name="Customer RFM Analysis")
df_second_page.head()

,NOTE - The original Recency calculation was incorrect. This worksheet has the corrected calculation.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9


In [29]:
df_second_page = df_second_page.dropna()
df_second_page.head()

,NOTE - The original Recency calculation was incorrect. This worksheet has the corrected calculation.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
1,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9
5,11003,2013-05-10 00:00:00,281,3,8139.29,2,9,9


In [30]:
column_names = df_second_page.columns.tolist()
print("Column Names:")
for column_name in column_names:
    print(column_name)

Column Names:
NOTE - The original Recency calculation was incorrect. This worksheet has the corrected calculation.
Unnamed: 1
Unnamed: 2
Unnamed: 3
Unnamed: 4
Unnamed: 5
Unnamed: 6
Unnamed: 7


In [31]:
new_column_names = [
    "CustomerID", "MostRecentOrderDate", "DaysSinceLastOrder",
    "SalesOrderCount", "TotalSalesAmount", "Recency", "Frequency", "Monetary"]
df_second_page.columns = new_column_names
df_second_page.head()

,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
1,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9
5,11003,2013-05-10 00:00:00,281,3,8139.29,2,9,9


In [32]:
df_second_page = df_second_page.drop(df_second_page.index[0])
df_second_page.head()

,CustomerID,MostRecentOrderDate,DaysSinceLastOrder,SalesOrderCount,TotalSalesAmount,Recency,Frequency,Monetary
2,11000,2013-05-03 00:00:00,288,3,8248.99,2,9,9
3,11001,2013-12-10 00:00:00,67,3,6383.88,8,9,9
4,11002,2013-02-23 00:00:00,357,3,8114.04,0,9,9
5,11003,2013-05-10 00:00:00,281,3,8139.29,2,9,9
6,11004,2013-05-01 00:00:00,290,3,8196.01,2,9,9


In [33]:
df_second_page['DaysSinceLastOrder'] = pd.to_numeric(df_second_page['DaysSinceLastOrder'], errors='coerce')
# Round and convert 'TotalSalesAmount' column to integers
df_second_page['TotalSalesAmount'] = df_second_page['TotalSalesAmount'].apply(lambda x: int(round(x)))


#df_second_page['TotalSalesAmount'] = df_second_page['TotalSalesAmount'].round().astype(int)


try:
  df_second_page['SalesOrderCount'] = pd.to_numeric(df_second_page['SalesOrderCount'], errors='coerce')
except ValueError:
  print("Warning: 'SalesOrderCount' column contains non-numeric values and cannot be converted to int.")


print(df_second_page.dtypes)



CustomerID             object
MostRecentOrderDate    object
DaysSinceLastOrder      int64
SalesOrderCount         int64
TotalSalesAmount        int64
Recency                object
Frequency              object
Monetary               object
dtype: object


In [65]:
def assign_score(value):
    if value >= 0.8:
        return 5
    elif value >= 0.6:
        return 4
    elif value >= 0.4:
        return 3
    elif value >= 0.2:
        return 2
    else:
        return 1

# Apply pd.qcut to bin values into percentile ranges and assign scores
for col in ['DaysSinceLastOrder','SalesOrderCount', 'TotalSalesAmount']:
    min_val = df_second_page[col].min()
    max_val = df_second_page[col].max()
    scaled_values = (df_second_page[col] - min_val) / (max_val - min_val)  # Perform min-max scaling
    percentiles = pd.qcut(df_second_page[col], 5, labels=False, duplicates='drop')  # Compute percentiles
    scores = percentiles.apply(lambda x: assign_score(x / 5))  # Map percentiles to scores
    df_second_page[col + '_score'] = scores  # Assign scores to new column

# Rename the new columns
df_second_page.rename(columns={'DaysSinceLastOrder_score': 'R2', 'SalesOrderCount_score': 'F2', 'TotalSalesAmount_score': 'M2'}, inplace=True)

# Find duplicated column names
duplicated_columns = df_second_page.columns[df_second_page.columns.duplicated()]

# Drop duplicated columns
df_second_page.drop(columns=duplicated_columns, inplace=True)
print(df_second_page)

   CustomerID  MostRecentOrderDate  DaysSinceLastOrder  SalesOrderCount  \
2       11000  2013-05-03 00:00:00                 288                5   
14      11012  2013-10-15 00:00:00                 123                3   
17      11015  2013-01-18 00:00:00                 393                1   

    TotalSalesAmount Recency Frequency Monetary  sum  R2  F2  M2  
2               8249       2         9        9    9   3   5   5  
14                81       6         6        3    7   1   3   1  
17              2501       0         0        7    6   5   1   3  


In [67]:
max_r2 = df_second_page['R2'].max()
max_f2 = df_second_page['F2'].max()
max_m2 = df_second_page['M2'].max()
print("Max value of R2:", max_r2)
print("Max value of M2:", max_m2)
print("Max value of F2:", max_f2)

Max value of R2: 5
Max value of M2: 5
Max value of F2: 5
